In [3]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

In [1]:
DATA_DIR = "../elliptic_dataset"

# wallet data
WALLETS_FEATURES = "wallets_features.csv"
WALLETS_CLASSES = "wallets_classes.csv"

# wallet to wallet edges
ADDRESS_TO_ADDRESS = "AddrAddr_edgelist.csv"


## 1. Load and inspect the actors dataset

#### 1.1.Let's first inspect the features of wallets

In [4]:
wallet_features = pd.read_csv(os.path.join(DATA_DIR, WALLETS_FEATURES))

In [4]:
print(wallet_features.shape[0])

1268260


In [5]:
wallet_features.head(5)

,address,Time step,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,111112TykSw72ztDN2WJger4cynzWYC5w,25,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.0,0.0,0.000000,0.0,0.0,24.0,1.0,1.0,1.0,1.0
1,1111DAYXhoxZx2tsRnzimfozo783x1yC2,25,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
2,1111DAYXhoxZx2tsRnzimfozo783x1yC2,29,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
3,1111DAYXhoxZx2tsRnzimfozo783x1yC2,39,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
4,1111DAYXhoxZx2tsRnzimfozo783x1yC2,39,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0


#### 1.2. Lets now look at the wallet labels:
- Class 1 - Illicit (should be 14,266, so 2%)
- Class 2 - Licit (should be 251,088, so 31%)
- Class 3 - Unknown (the rest)

In [6]:
wallet_classes = pd.read_csv(os.path.join(DATA_DIR, WALLETS_CLASSES))

In [7]:
print(wallet_classes.shape[0])

822942


In [8]:
wallet_classes.head(5)

,address,class
0,111112TykSw72ztDN2WJger4cynzWYC5w,2
1,1111DAYXhoxZx2tsRnzimfozo783x1yC2,3
2,1111VHuXEzHaRCgXbVwojtaP7Co3QABb,2
3,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,3
4,1115LWW3xsD9jT9VRY7viCN9S34RVAAuA,2


Let's verify the counts

In [10]:
illicit_count = wallet_classes[wallet_classes["class"] == 1].shape[0]
licit_count = wallet_classes[wallet_classes["class"] == 2].shape[0]
unknown_count = wallet_classes[wallet_classes["class"] == 3].shape[0]

print(f"Illicit transaction count: {illicit_count}")
print(f"Licit transaction count: {licit_count}")
print(f"Unknown transaction count: {unknown_count}")

Illicit transaction count: 14266
Licit transaction count: 251088
Unknown transaction count: 557588


Ok so the illicit and licit counts check out. When populating over time we will see that the counts should match up ot the total of wallet_features

#### 1.3. Let's inspect the address to address edges

In [11]:
address_to_address = pd.read_csv(os.path.join(DATA_DIR, ADDRESS_TO_ADDRESS))

In [12]:
print(address_to_address.shape[0])

2868964


In [13]:
address_to_address.head(5)

,input_address,output_address
0,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,1GASxu5nMntiRKdVtTVRvEbP965G51bhHH
1,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a
2,13Lhad3SAmu2vqYg2dxbNcxH7LE77kJu2w,1GFdrdgtG34GChM8SMpMwcXFc4nYbH1A5G
3,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,19q57SeCEzTnWrWVXA43nZzhSiXkYggh7c
4,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,1Kk1NVYnCE8ALXDhgMM6HqTt1jDSvi6QBA


Ok this is not enough, we want to have some edge features. Lets build this ourselves and include transaction features such as amount etc.

## 2. Prepare an address-address transaction list with extra features (amount time etc)

In [25]:
TRANSACTION_FEATURES = "txs_features.csv"
TRANSACTION_CLASSES = "txs_classes.csv"
TRANSACTION_EDGE_LIST = "txs_edgelist.csv"
ADDRESES_TO_TRANSACTIONS = "AddrTx_edgelist.csv"
TRANSACTIONS_TO_ADDRESSES = "TxAddr_edgelist.csv"

transaction_edges = pd.read_csv(os.path.join(DATA_DIR, TRANSACTION_EDGE_LIST))
transaction_features = pd.read_csv(os.path.join(DATA_DIR, TRANSACTION_FEATURES))
transaction_classes = pd.read_csv(os.path.join(DATA_DIR, TRANSACTION_CLASSES))
address_transaction = pd.read_csv(os.path.join(DATA_DIR, ADDRESES_TO_TRANSACTIONS))
transaction_address = pd.read_csv(os.path.join(DATA_DIR, TRANSACTIONS_TO_ADDRESSES))

In [26]:
transaction_edges.head(5)

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [15]:
transaction_features.shape, transaction_classes.shape

((203769, 184), (203769, 2))

In [16]:
key_transaction_cols = ['txId', 'Time step', 'in_txs_degree', 'out_txs_degree', 'total_BTC', 
                        'fees', 'size', 'num_input_addresses', 'num_output_addresses',
                        'in_BTC_total', 'out_BTC_total']
                        
transaction_features[key_transaction_cols].head()

,txId,Time step,in_txs_degree,out_txs_degree,total_BTC,fees,size,num_input_addresses,num_output_addresses,in_BTC_total,out_BTC_total
0,3321,1,1.0,0.0,0.533972,0.0001,225.0,1.0,2.0,0.534072,0.533972
1,11108,1,1.0,1.0,5.611778,0.0001,225.0,1.0,2.0,5.611878,5.611778
2,51816,1,1.0,1.0,0.456508,0.0001,226.0,1.0,2.0,0.456608,0.456508
3,68869,1,0.0,1.0,9.308800,0.0001,853.0,3.0,2.0,9.308900,9.308800
4,89273,1,1.0,288.0,852.164680,0.0000,445268.0,1.0,13107.0,852.164680,852.164680


In [17]:
address_transaction.head()

,input_address,txId
0,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,230325127
1,13Lhad3SAmu2vqYg2dxbNcxH7LE77kJu2w,230325139
2,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,86875675
3,16zs5SVSyADh5WrLNbZbpRLsBsN5uEzgeK,230325147
4,1QJpwtUorBKPGUJkSyrRcBKTAHq4CXrdYh,230325154


Build the complete address-to-address transaction dataframe

In [19]:
# merge address_transaction with transaction_features
address_transaction_with_features = address_transaction.merge(
    transaction_features, 
    on='txId', 
    how='left'
)

# merge with transaction_address to get output addresses
address_address_transactions = address_transaction_with_features.merge(
    transaction_address, 
    on='txId', 
    how='inner'
)

# merge with transaction classes (we might want to drop that later)
address_address_transactions = address_address_transactions.merge(
    transaction_classes, 
    on='txId', 
    how='left'
)

# sort index
address_address_transactions['class'].value_counts().sort_index()
address_address_transactions.shape

(2868964, 187)

We can see that the number of entries is same as for address-address. This checks out. We also have more entries than transaction features - thsi is because each BTC transaction can include mutliple actors. So a transaction with N inputs and M outputs will have NxM entries added to addresse_address_transactions table.

In [20]:
address_address_transactions.groupby('Time step').size().describe()

count        49.000000
mean      58550.285714
std       40317.169123
min        9932.000000
25%       26861.000000
50%       59974.000000
75%       73202.000000
max      190334.000000
dtype: float64

Let's have a look if the BTC amount features differ - also the amounts are misleading as this is a total amount of transactions, and those could include multiple addresses and a bunch of sub-flows -w e coudl scrape this data (painful) on rely on diff features or maybe normalize by using the num_***_addresses features.

In [21]:
address_address_transactions[['total_BTC', 'in_BTC_total', 'out_BTC_total']].describe()

,total_BTC,fees,in_BTC_total,out_BTC_total
count,2.868964e+06,2.868964e+06,2.868964e+06,2.868964e+06
mean,5.197374e+01,1.602468e-02,5.198976e+01,5.197374e+01
std,1.675581e+02,2.649537e-02,1.675601e+02,1.675581e+02
min,6.000000e-06,0.000000e+00,4.127000e-05,6.000000e-06
25%,2.488004e-01,7.532000e-04,2.501565e-01,2.488004e-01
50%,5.207729e+00,2.300000e-03,5.222652e+00,5.207729e+00
75%,3.242890e+01,2.451313e-02,3.242940e+01,3.242890e+01
max,1.139186e+04,2.851285e-01,1.139186e+04,1.139186e+04


Prepare an aggregated variant where all transactions between nodes are aggregated. This is suboptimal because we lose the temporal information - however maybe some other grouping into time steps is still posssible based on this. 

In [27]:
# Aggregate by address pair (optional - one row per address pair instead of per transaction)
address_pair_aggregated = address_address_transactions.groupby(['input_address', 'output_address']).agg({
    'txId': 'count',
    'total_BTC': ['sum', 'mean', 'median', 'min', 'max'],
    'fees': ['sum', 'mean'],
    'size': ['sum', 'mean'],
    'Time step': ['min', 'max', 'nunique'],
    'class': lambda x: x.mode()[0] if len(x.mode()) > 0 else 3
}).reset_index()

address_pair_aggregated.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                                    for col in address_pair_aggregated.columns.values]

address_pair_aggregated.head(10)

,input_address,output_address,txId_count,total_BTC_sum,total_BTC_mean,total_BTC_median,total_BTC_min,total_BTC_max,fees_sum,fees_mean,size_sum,size_mean,Time step_min,Time step_max,Time step_nunique,class_<lambda>
0,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,1A2vTkKSsmVLN2EPEJT3KZR4q1Rvv6c6Xs,1,0.056435,0.056435,0.056435,0.056435,0.056435,0.000246,0.000246,521.0,521.0,17,17,1,3
1,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,1KWbPoFkzadegdff9rCK1wBFu3mD8M17Wp,1,0.056435,0.056435,0.056435,0.056435,0.056435,0.000246,0.000246,521.0,521.0,17,17,1,3
2,1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ,1K7o3aMfiddvUgMGagdNE5GkiykPPyGj32,1,1.214091,1.214091,1.214091,1.214091,1.214091,0.000100,0.000100,521.0,521.0,5,5,1,2
3,1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ,1Po4J4SNyJuGnMGYJfGTXLEvGgAZKiddr7,1,1.214091,1.214091,1.214091,1.214091,1.214091,0.000100,0.000100,521.0,521.0,5,5,1,2
4,111HRAJxnoxqyKRVnjqBmwqneUrHc1chi,12RoZAgmZMFHMMrvaqrYZrLMPpAFEFGyWU,1,3.013936,3.013936,3.013936,3.013936,3.013936,0.002322,0.002322,4204.0,4204.0,23,23,1,3
5,111HRAJxnoxqyKRVnjqBmwqneUrHc1chi,157QmRJdktRBe99EKE5gYYBY8m9bdBwXuE,1,3.013936,3.013936,3.013936,3.013936,3.013936,0.002322,0.002322,4204.0,4204.0,23,23,1,3
6,111Ud9zxFi3VgcstEM2R3YAS3CNxf9o8r,19eJN6g8dJFubasMXm5UYFC6jfYx3nDKYV,1,129.142504,129.142504,129.142504,129.142504,129.142504,0.028206,0.028206,18224.0,18224.0,33,33,1,2
7,111Ud9zxFi3VgcstEM2R3YAS3CNxf9o8r,1KPijPK8rTYZCdAP3JmsjVnXftrdnAfopf,1,129.142504,129.142504,129.142504,129.142504,129.142504,0.028206,0.028206,18224.0,18224.0,33,33,1,2
8,111Y3BkUGLSWQjHCb2Mg8oMNiwc1jxMwe,12eweZVTQx8MktqkEodiss1z2RqCbYjDqE,1,29.642631,29.642631,29.642631,29.642631,29.642631,0.029089,0.029089,13203.0,13203.0,36,36,1,2
9,111Y3BkUGLSWQjHCb2Mg8oMNiwc1jxMwe,15sFeTg3hB3fuCCT9ZZMX4jmbNqxfm2ZQ1,1,18.706521,18.706521,18.706521,18.706521,18.706521,0.017004,0.017004,7457.0,7457.0,36,36,1,2


Now let's see how many transactions do we have between different addresses across the dataset.

In [24]:
address_pair_transaction_counts = address_address_transactions.groupby(['input_address', 'output_address']).size()

print(f"Pairs with 1 transaction: {(address_pair_transaction_counts == 1).sum():,}")
print(f"Pairs with 2-5 transactions: {((address_pair_transaction_counts >= 2) & (address_pair_transaction_counts <= 5)).sum():,}")
print(f"Pairs with 6-10 transactions: {((address_pair_transaction_counts >= 6) & (address_pair_transaction_counts <= 10)).sum():,}")
print(f"Pairs with >10 transactions: {(address_pair_transaction_counts > 10).sum():,}")
print(f"Max transactions for single pair: {address_pair_transaction_counts.max()}")

Pairs with 1 transaction: 2,755,799
Pairs with 2-5 transactions: 26,693
Pairs with 6-10 transactions: 1,244
Pairs with >10 transactions: 608
Max transactions for single pair: 1453


Let's have a look at all the features now:

In [28]:
address_address_transactions.head(5)

,input_address,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,...,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,output_address,class
0,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,230325127,1,-0.128834,0.048298,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,7.000303,7.000303,7.000303,0.173495,6.825808,3.499652,3.499652,6.999303,1GASxu5nMntiRKdVtTVRvEbP965G51bhHH,3
1,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,230325127,1,-0.128834,0.048298,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,7.000303,7.000303,7.000303,0.173495,6.825808,3.499652,3.499652,6.999303,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,3
2,13Lhad3SAmu2vqYg2dxbNcxH7LE77kJu2w,230325139,1,-0.138128,-0.184668,-1.201369,0.028105,-0.063725,-0.113002,0.547008,...,1.841967,1.194726,5.525902,5.525802,5.525802,5.525802,5.525802,5.525802,1GFdrdgtG34GChM8SMpMwcXFc4nYbH1A5G,3
3,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,86875675,1,-0.098483,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,...,3.937091,3.852260,11.811274,1.266853,10.544321,5.905587,5.905587,11.811174,19q57SeCEzTnWrWVXA43nZzhSiXkYggh7c,3
4,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,86875675,1,-0.098483,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,...,3.937091,3.852260,11.811274,1.266853,10.544321,5.905587,5.905587,11.811174,1Kk1NVYnCE8ALXDhgMM6HqTt1jDSvi6QBA,3


Prepeare a version iwth the known subset of features

In [29]:
address_address_transactions_known_features = address_address_transactions.loc[
    :, ~address_address_transactions.columns.str.startswith(("Aggregate_feature", "Local_feature"))
]

In [30]:
address_address_transactions_known_features.head(5)

,input_address,txId,Time step,in_txs_degree,out_txs_degree,total_BTC,fees,size,num_input_addresses,num_output_addresses,...,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,output_address,class
0,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,230325127,1,0.0,1.0,6.999303,0.0010,225.0,1.0,2.0,...,7.000303,7.000303,7.000303,0.173495,6.825808,3.499652,3.499652,6.999303,1GASxu5nMntiRKdVtTVRvEbP965G51bhHH,3
1,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,230325127,1,0.0,1.0,6.999303,0.0010,225.0,1.0,2.0,...,7.000303,7.000303,7.000303,0.173495,6.825808,3.499652,3.499652,6.999303,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,3
2,13Lhad3SAmu2vqYg2dxbNcxH7LE77kJu2w,230325139,1,0.0,1.0,5.525802,0.0001,486.0,3.0,1.0,...,1.841967,1.194726,5.525902,5.525802,5.525802,5.525802,5.525802,5.525802,1GFdrdgtG34GChM8SMpMwcXFc4nYbH1A5G,3
3,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,86875675,1,0.0,1.0,11.811174,0.0001,521.0,3.0,2.0,...,3.937091,3.852260,11.811274,1.266853,10.544321,5.905587,5.905587,11.811174,19q57SeCEzTnWrWVXA43nZzhSiXkYggh7c,3
4,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,86875675,1,0.0,1.0,11.811174,0.0001,521.0,3.0,2.0,...,3.937091,3.852260,11.811274,1.266853,10.544321,5.905587,5.905587,11.811174,1Kk1NVYnCE8ALXDhgMM6HqTt1jDSvi6QBA,3


Write to csv - the edgelist is divided into 8 parts so that LFS doesnt bitch abt it

In [33]:
def split_into_n_parts(df, n, out_dir, base):
    parts = np.array_split(df, n)
    paths = []
    for i, part in enumerate(parts, 1):
        p = os.path.join(out_dir, f"{base}_part_{i}.csv")
        part.to_csv(p, index=False)

split_into_n_parts(address_address_transactions, n=8, out_dir=DATA_DIR, base="AddrTxAddr_edgelist")

/Users/franciszeklatala/mamba/envs/graph-ml/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


The rest is good to go at once

In [ ]:
# address_address_transactions.to_csv(os.path.join(DATA_DIR, "AddrTxAddr_edgelist.csv"), index=False)
address_pair_aggregated.to_csv(os.path.join(DATA_DIR, "AddrTxAddr_edgelist_aggregated.csv"), index=False)
address_address_transactions_known_features.to_csv(os.path.join(DATA_DIR, "AddrTxAddr_edgelist_known_features.csv"), index=False)